# 降维与度量学习

当训练集中样本本身的属性维度过高时，要实现密采样条件是很难的（就是针对每个属性，都可以在足够小的 $\delta$ 距离找到足够多的样本）,同时许多学习方法都涉及到距离计算，高维空间的距离计算是很麻烦的，甚至内积计算都不再容易。

事实上，高维情形下出现的数据样本稀疏、距离计算困难等问题，是所有机器学习方法共同面临的严重障碍，被称为“维数灾难”。

**缓解维数灾难的一个重要途径是降维，也叫“维数约简”（另一个重要途径是特征选择），即通过某种数学变换将原始高维属性空间转变为一个低维“子空间”，在这个子空间上样本密度大幅提高，距离计算也变得容易。**

**为什么可以降维？因为很多时候，人们收集到的数据样本虽然是高维的，但与学习任务密切相关的仅是某个低维分布，即高维空间的一个低维“嵌入”。**

## 多维缩放(MDS)

### 问题的提出

如果要求这个数学变换是**保距映射，即样本之间的距离在低维空间得以保持，**就是我们本节要介绍的**多维缩放(Multiple Dimensional Scaling，简称 MDS)**。

给定数据集 $D = \{ (\mathbf{x}_1, y_1), (\mathbf{x}_2, y_2), ... , (\mathbf{x}_m, y_m)\}$ 包含 m 个样本，其中 $ \mathbf{x}_i = (x_{i1}, x_{i2}, ...,x_{id})$ 为 d 维的样本。对应的自变量为 $S = (\mathbf{x}_1, \mathbf{x}_2,...,\mathbf{x}_m)\in R^{d\times m}$。

假设 m 个样本在原始空间的距离矩阵是 

$$\mathbf{D} = (d_{ij})\in R^{m\times m}$$, 

其中$d_{ij} = dist(\mathbf{x}_i,\mathbf{x}_j)$ 是两个样本之间的距离。

我们的目标是**获得降维后的自变量$ S_1 =  (\mathbf{z}_1,\mathbf{z}_2, ... , \mathbf{z}_m)$,
其中 $\mathbf{z}_i = (z_{i1}, z_{i2}, ...,z_{id_1})$ 为 $d_1\; (d_1 \leq d)$ 维度的样本。且**

$$ ||\mathbf{z}_i - \mathbf{z}_j|| = d_{ij}$$

假定 $S_1$ 对应的数据集为 $D_1 = \{ (\mathbf{z}_1, y_1), (\mathbf{z}_2, y_2), ... , (\mathbf{z}_m, y_m)\}$。

也就是说，我们的目标是**寻找一个数学变换使得原来的数据集 $D\in R^{(d+1)\times m}$ 保距离映射到数据集 $D_1\in R^{(d_1+1)\times m}, d_1 \leq d$。**

### 解决思路

令 $\mathbf{B} = \mathbf{S}_1^T \mathbf{S}_1 = (\mathbf{z}_i^T \mathbf{z}_j)\in R^{m\times m}$, 其中 $\mathbf{B}$ 为降维后样本的**内积矩阵**， $b_{ij} = \mathbf{z}_i^T \mathbf{z}_j$，因为这个变换是保距的，所有有

$$\begin{split}
d_{ij}^2 &= ||\mathbf{z}_i - \mathbf{z}_j||^2\\
&= ||\mathbf{z}_i||^2 + ||\mathbf{z}_j||^2 - 2\mathbf{z}_i^T \mathbf{z}_j\\
&= b_{ii}+b_{jj}-2b_{ij}
\end{split}$$

也因此我们有
$$ b_{ij} = -\frac{1}{2}(d_{ij}^2 - b_{ii}-b_{jj}) \tag{MDS-1}$$

**step1. 下面我们先通过原空间的距离矩阵 D 来求解满足保距关系 (MDS-1) 的距离矩阵 B.**

为便于讨论，令降维后的 $\mathbf{S}_1$ 被中心化，即 $\sum_{i=1}^m \mathbf{z}_i = \mathbf{0}$.显然，矩阵 $\mathbf{B}$ 的行/列之和均为0，即 

$$\sum_{i=1}^m b_{ij} = \left(\sum_{i=1}^m \mathbf{z}_i\right)^T \mathbf{z}_j= 0$$

等价地，也有 $\sum_{j=1}^m b_{ij}=0$.

由 (MDS-1) 式可知

$$\sum_{i=1}^m d_{ij}^2 = \sum_{i=1}^m b_{ii} + \sum_{i=1}^m b_{jj} - 2 \sum_{i=1}^m b_{ij}= tr(\mathbf{B}) + m b_{jj} \tag{MDS-2.1}$$

其中，$tr(\cdot)$ 表示矩阵的迹(trace)，$tr(\mathbf{B}) = \sum_{i=1}^m ||\mathbf{z}_i||^2$。

类似地

$$\sum_{j=1}^m d_{ij}^2 = tr(\mathbf{B}) + m b_{ii} \tag{MDS-2.2}$$
$$\sum_{i=1}^m\sum_{j=1}^m d_{ij}^2 = 2m tr(\mathbf{B})\tag{MDS-2.2}$$

令

$$d_{i\cdot}^2 = \frac{1}{m}\sum_{j=1}^m d_{ij}^2 \tag{MDS-3.1}$$

$$d_{\cdot j}^2 = \frac{1}{m}\sum_{i=1}^m d_{ij}^2 \tag{MDS-3.2}$$

$$d_{\cdot\cdot}^2 = \frac{1}{m^2}\sum_{i=1}^m\sum_{j=1}^m d_{ij}^2 \tag{MDS-3.3}$$


由 (MDS-1) 和 MDS-$(2.1\sim 3.3)$ 可得

$$\begin{split}
b_{ij} &= -\frac{1}{2}(d_{ij}^2 - b_{ii}-b_{jj}) \\
&= -\frac{1}{2}\left(d_{ij}^2 - \frac{1}{m}\sum_{i=1}^m d_{ij}^2 -\frac{1}{m}tr(\mathbf{B}) -\frac{1}{m}\sum_{j=1}^m d_{ij}^2 -\frac{1}{m}tr(\mathbf{B})\right)\\
&= -\frac{1}{2}\left(d_{ij}^2 - d_{\cdot j}^2 - d_{i\cdot}^2 - \frac{2}{m}\frac{1}{2m}\sum_{i=1}^m\sum_{j=1}^m d_{ij}^2\right)\\
&= -\frac{1}{2}(d_{ij}^2 - d_{\cdot j}^2 - d_{i\cdot}^2 - d_{\cdot\cdot}^2)
\end{split}\tag{MDS-4}$$

到此为止，我们得到了降维后保持样本之间距离不变的内积矩阵 B。

**step2. 通过内积矩阵 B 求解 $S_1$ 的表达形式。**

对矩阵做特征值分解

$$\mathbf{B} = \mathbf{V}\mathbf{\Lambda}\mathbf{V}^T$$

其中 $\Lambda  = diag(\lambda_1, \lambda_2, ..., \lambda_d)$ 为特征值构成的对角矩阵，$\lambda_1\geq \lambda_2\geq ...,\geq\lambda_d,\; \mathbf{V}$ 为特征向量矩阵。

假定其中有 $d^\ast$ 个非零特征值，构成对角矩阵 $\Lambda_\ast = diag(\lambda_1, \lambda_2, ..., \lambda_{d_\ast})$, 令 $\mathbf{V}_\ast$ 表示相应的特征向量矩阵，则 $S_1$ 可表示为

$$ S_1 = \Lambda_\ast^{1/2}\mathbf{V}_\ast^T \in R^{d^\ast\times m}.$$

到此为止，我们得到了降维后对应的 $S_1$ 的表达形式。

现实应用中，往往只需要降维后的距离与原始空间距离尽可能接近，不要求严格相等。此时可取 $d_1 \ll d$ 个最大特征值构成对角矩阵 $\tilde{\Lambda} = diag(\lambda_1, \lambda_2, ..., \lambda_{d_1})$，令$\tilde{\mathbf{V}}$ 表示相应的特征向量矩阵，则 $S_1$ 可表达为

$$S_1 = \tilde{\Lambda}^{1/2}\tilde{\mathbf{V}}^T \in R^{d_1\times m}.$$

### 算法流程图

下面给出 MDS 的算法流程图：
***
* **输入：** 距离矩阵 $\mathbf{D} = (d_{ij})\in R^{m\times m}$;  低维空间维数 $d_1$.


* **过程：**
    1. 根据式 MDS-$(3.1\sim 3.3)$ 计算 $d_{i\cdot}^2,\; d_{\cdot j}^2,\; d_{\cdot\cdot}^2$;
    2. 根据式 (MDS-4) 计算内积矩阵 $\mathbf{B}$;
    3. 对矩阵 $\mathbf{B}$ 做特征分解；
    4. 取 $\tilde{\Lambda}$ 为 $d_1$ 个最大特征值所构成的对角矩阵， $\tilde{\mathbf{V}}$ 为相应的特征向量矩阵.
    
    
* **输出：** 矩阵 $\tilde{\mathbf{V}}\tilde{\Lambda}^{1/2}\in R^{m\times d_1}$,每行是一个样本的低维坐标($d_1$ 维)

疑问： 输入里的距离矩阵本身在高维空间好求解么？？
***

### [sklearn.manifold.MDS](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html)

In [1]:
# 小案例
from sklearn.datasets import load_digits
from sklearn.manifold import MDS
# 加载数据
X, _ = load_digits(return_X_y=True)
X.shape

(1797L, 64L)

In [4]:
embedding = MDS(n_components=2)
X_transformed = embedding.fit_transform(X[:100])
X_transformed.shape

(100L, 2L)

## 流形学习

“流形”是微分几何里的概念，简单来讲，是指在局部与欧式空间通胚的空间。给降维方法带来的启发是：

    若低维流形嵌入到高维空间中，则数据样本在高维空间的分布虽然看上去非常复杂，但在局部上仍具有欧式空间的性质，因此，可以容易地在局部建立降维映射关系，再设法将局部映射关系推广到全局。

### 等度量映射（Isomap）

等度量映射的基本出发点是：

    任务低维流形嵌入到高维空间后，直接在高维空间中计算直线距离具有误导性，因为高维空间中的直线距离在低维嵌入流形是不可达的。低维嵌入流形两点间的距离是“测地线”距离。
    
    通过“在局部上与欧式空间同胚”的性质，对每个点在基于欧式距离找出其近邻点，然后建立起一个近邻连接图，近邻点之间存在连接，非近邻点之间不存在连接。于是，计算两点之间测地线距离的问题，转变为计算近邻连接图上两点之间最短路径问题。在得到任意两点的距离后，就可以通过上面介绍的 MDS 来获得样本点在低维空间中的坐标。

具体再详细的部分请参看周志华的《机器学习》第10章。对应的sklearn 包为 [sklearn.manifold.Isomap](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.Isomap.html#sklearn.manifold.Isomap)

### 局部线性嵌入（Locally Linear Embedding，简称 LLE）

与 Isomap 试图保持近邻样本之间的距离不同，局部线性嵌入**试图保持邻域内样本之间的线性关系（相当于说，在流形的局部我们假设样本的邻域是一个超平面，这个是微分里以直代曲的思想）**。

具体再详细的部分请参看周志华的《机器学习》第10章。对应的sklearn 包为 [sklearn.manifold.LocallyLinearEmbedding](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.LocallyLinearEmbedding.html#sklearn.manifold.LocallyLinearEmbedding)

## 度量学习

度量学习的基本动机：

    机器学习里，对高维数据进行降维的主要目的是希望找到一个合适的低维空间，在此空间中进行学习能比原始空间性能更好。事实上，**每个空间对应了在样本属性上定义的一个距离度量，寻找合适的空间，实质上就是寻找一个合适的距离度量。**

    那么，为何不直接尝试“学习”出一个合适的距离度量呢？

欧式距离的前提是，坐标轴是正交的，属性之间是不相关的。我们打破这两点，定义如下的马氏距离：

$$dist_{mah}^2(\mathbf{x}_i,\mathbf{x}_j) = (\mathbf{x}_i-\mathbf{x}_j)^T\mathbf{M}(\mathbf{x}_i-\mathbf{x}_j)=||\mathbf{x}_i-\mathbf{x}_j||^2_{\mathbf{M}}$$

其中 $\mathbf{M}$ 也被称为“度量矩阵”。度量学习就是对 $\mathbf{M}$ 的学习。

注意到为了保持距离非负且对称， $\mathbf{M}$ 必须是（半）正定对称矩阵，即必有正交基 $\mathbf{P}$ 使得 $\mathbf{M}$ 能写为

$$\mathbf{M} = \mathbf{P}\mathbf{P}^T$$

具体再详细的部分请参看周志华的《机器学习》第10章。